In [1]:
from heapq import heappush, heappop

class Task:
    def __init__(self, name, duration, dependencies=None, priority=None):
        self.name = name
        self.duration = duration
        self.dependencies = dependencies if dependencies else []
        self.priority = priority

    def __repr__(self):
        return self.name

def heuristic(task, remaining_tasks, use_priority=False):

    if not remaining_tasks:
        return 0
    if use_priority:
        return sum(t.duration / (t.priority if t.priority else 1) for t in remaining_tasks)
    else:
        return sum(t.duration for t in remaining_tasks)

def get_dependent_tasks(task, all_tasks):
   
    dependent_tasks = []
    for t in all_tasks:
        if task.name in t.dependencies:
            dependent_tasks.append(t)
    return dependent_tasks

def a_star(tasks, start_task, use_priority=False):
    
    open_set = []
    heappush(open_set, (0, start_task)) 
    g_score = {task: float('inf') for task in tasks}
    g_score[start_task] = 0
    came_from = {}

    while open_set:
        current_f, current_task = heappop(open_set)

        if not get_dependent_tasks(current_task, tasks):
            return reconstruct_path(came_from, current_task, g_score)

        for neighbor in get_dependent_tasks(current_task, tasks):
            tentative_g_score = g_score[current_task] + neighbor.duration

            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_task
                g_score[neighbor] = tentative_g_score
                remaining_tasks = get_dependent_tasks(neighbor, tasks)
                f_score = tentative_g_score + heuristic(neighbor, remaining_tasks, use_priority)
                heappush(open_set, (f_score, neighbor))

    return None

def reconstruct_path(came_from, current_task, g_score):
    
    total_time = g_score[current_task]
    path = []
    while current_task in came_from:
        path.append(current_task)
        current_task = came_from[current_task]
    path.append(current_task)
    path.reverse()
    return path, total_time

def greedy_schedule(tasks, start_task, use_priority=False):
   
    scheduled = []
    remaining_tasks = tasks.copy()
    current_task = start_task
    total_time = 0

    while remaining_tasks:
        scheduled.append(current_task)
        total_time += current_task.duration
        remaining_tasks.remove(current_task)

        next_task = None
        min_heuristic = float('inf')
        for task in remaining_tasks:
            if all(dep in scheduled for dep in task.dependencies):
                h = heuristic(task, get_dependent_tasks(task, remaining_tasks), use_priority)
                if h < min_heuristic:
                    min_heuristic = h
                    next_task = task

        if not next_task:
            break
        current_task = next_task

    return scheduled, total_time

tasks1 = [
    Task('T1', 10),
    Task('T2', 20, ['T1']),
    Task('T3', 55, ['T1']),
    Task('T4', 28, ['T3']),
    Task('T5', 40, ['T3', 'T4']),
]
tasks2 = [
    Task('T1', 10, priority=5),
    Task('T2', 20, ['T1'], priority=3),
    Task('T3', 55, ['T1'], priority=6),
    Task('T4', 28, ['T3'], priority=2),
    Task('T5', 40, ['T3', 'T4'], priority=2),
]

print("Task 1: Without Priorities")
a_star_path, a_star_time = a_star(tasks1, tasks1[0])
greedy_path, greedy_time = greedy_schedule(tasks1, tasks1[0])
print(f"A* Schedule: {a_star_path}, Total Time: {a_star_time}")
print(f"Greedy Schedule: {greedy_path}, Total Time: {greedy_time}")

print("\nTask 2: With Priorities")
a_star_path, a_star_time = a_star(tasks2, tasks2[0], use_priority=True)
greedy_path, greedy_time = greedy_schedule(tasks2, tasks2[0], use_priority=True)
print(f"A* Schedule: {a_star_path}, Total Time: {a_star_time}")
print(f"Greedy Schedule: {greedy_path}, Total Time: {greedy_time}")

Task 1: Without Priorities
A* Schedule: [T1, T2], Total Time: 20
Greedy Schedule: [T1], Total Time: 10

Task 2: With Priorities
A* Schedule: [T1, T2], Total Time: 20
Greedy Schedule: [T1], Total Time: 10
